# Welcome

Authors:
- Célien Donzé, research assistant at Haute Ecole Arc Ingénierie, Switzerland
- Jonathan Guerne, research assistant at Haute Ecole Arc Ingénierie, Switzerland
- Henrique Marques Reis, research assistant at Haute Ecole Arc Ingénierie, Switzerland
- Pedro Costa, CO-Founder and CTO at Lumind, Switzerland

## Package installation

In [46]:
!pip install langchain langchain-community faiss-cpu pymupdf pypdf sentence_transformers rich wget

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Imports

In [47]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms.ollama import Ollama
import os
import json
from rich.console import Console
import zipfile
import os
import wget


console=Console()

# Downloading the pdfs

In [48]:
# Create the "data/PDFs" folder if it doesn't exist
os.makedirs("data/PDFs", exist_ok=True)

# Download the PDFs
url = "https://www.dropbox.com/scl/fo/xhqjzofiqnbmraxksgvlh/AAoL_WMBFOYDuipk5T_tTus?rlkey=qbbcvw4gbw6bpxkeijt6m94kt&st=yhap82wh&dl=1"
filename=wget.download(url, ".")

zip_file_path = f"./{filename}"
extract_folder = "data/PDFs"

# Create the extract folder if it doesn't exist
os.makedirs(extract_folder, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print("Pdf file downloaded successfully.")

Pdf file downloaded successfully.


## Documentation

- [langchain](https://python.langchain.com/v0.1/docs/get_started/introduction/)
- [Ollama website](https://ollama.com/)

## Constants

In [49]:
OLLAMA_ADDRESS = "http://157.26.83.14:11434"
# OLLAMA_ADDRESS = "http://165.1.69.123:11434"
LLM_NAME="gemma"

# start step 1

## Connecting to LLM

In [50]:
llm= Ollama(
        model=LLM_NAME,
        base_url=OLLAMA_ADDRESS,
        temperature=0.1, # Will be explained later
        stop=["<end_of_turn>"],
    )

## Creating a prompt

A prompt is generally divided into two parts: the context and the question.

The context provides the information that the model will use to generate its answer, while the question specifies what the model is expected to respond to.

In a prompt, special characters are used to delineate different sections. For instance, in Gemma, these are `<start_of_turn>` and `<end_of_turn>`.

Additionally, LangChain requires markers indicating where to insert the user's question and the context retrieved from documents. For the question.

Gemma prompt template :

```html
<start_of_turn>user
{{ if .System }}{{ .System }} {{ end }}{{ .Prompt }}<end_of_turn>
<start_of_turn>model
{{ .Response }}<end_of_turn>
```

In [51]:
template = """<start_of_turn>
You are an helpful assistant that answer the question in detail.

Human input: {question}<end_of_turn>
<start_of_turn>Assistant:<end_of_turn>"""

prompt = PromptTemplate(input_variables=["question"], template=template)

## Creating the chain and start a conversation

In [52]:
conversation = LLMChain(
    llm=llm,
    # verbose=True, # uncomment if you want to see more information about the chain
    prompt=prompt
)

In [53]:
result = conversation.invoke(input="What is the capital of Switzerland?")
console.print(result.get("text"))

**The capital of Switzerland is Bern.**

**Background:**

* Bern is a city in western Switzerland and has been the federal capital of Switzerland since 1848.
* It is located on the Aare River in the German-speaking region of the country.
* Bern is known for its charming Old Town, medieval architecture, and picturesque riverfront promenades.

**Key Features:**

* **Parliamentary Building:** The seat of the Swiss Federal Assembly, the highest legislative body in Switzerland.
* **Federal Palace:** The executive branch of the Swiss government.
* **Zytglogge:** A historic clock tower and symbol of the city.
* **Einstein Museum:** Dedicated to the life and work of Albert Einstein, who lived in Bern for many years.
* **Rosengartens:** A beautiful rose garden with over 10,000 roses.

**Transportation:**

* Bern is well-connected by public transport, including buses, trams, and trains.
* The city has a central train station with connections to other major cities in Switzerland.

**Accommodation:**

* Bern offers a wide range of accommodation options, from budget-friendly hotels to luxury apartments.
* Popular hotels include the Hotel Schweizerhof Bern and the Hotel Rössli.

**Attractions:**

* Explore the Old Town with its narrow streets and medieval buildings.
* Visit the Bundeshaus, the Swiss Parliament building.
* Take a stroll along the Aare River.
* Enjoy panoramic views from the Harder Kulm mountain.

**Accessibility:**

* Bern is easily accessible by train, bus, or plane.
* The city is flat and easy to navigate on foot or by public transport.

# end step 1

# start step 2

## Loading a PDF

In [54]:
DATA_DIR = os.path.join("./", "data")
PDF_DIR = os.path.join(DATA_DIR, "PDFs")
VECTORSTORES_DIR = os.path.join(DATA_DIR, "vectorstores")

In [55]:
loader = PyPDFDirectoryLoader(PDF_DIR)
doc = loader.load()

## Embedding a PDF in a vectorstore

In [56]:
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
EMBEDDING_MODEL_NAME = "BAAI/bge-large-en-v1.5"

<div>
<img src="chunk_overlap_size_scheme.png" width="800"/>
</div>

In [57]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDING_MODEL_NAME, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/mnt/d/dev/rag-workshop/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [58]:
all_splits = text_splitter.split_documents(doc)
vectorstore = FAISS.from_documents(documents=all_splits,embedding=embedding_model)


In [59]:
vectorstore.save_local(VECTORSTORES_DIR)

# end step 2

# start step 3

## Loading a vectorstore

In [60]:
vectorstore = FAISS.load_local(VECTORSTORES_DIR,embedding_model,allow_dangerous_deserialization=True)

## What is temperature?

The temperature parameter in a language model (LLM) controls the randomness of the model's output.

A lower temperature value (closer to 0) makes the model more deterministic, favoring higher probability words and resulting in more predictable and repetitive text.

A higher temperature value (closer to 1) increases randomness, allowing for more creative and diverse responses by giving less probable words a better chance of being chosen.

Adjusting the temperature helps balance between coherence and creativity in the generated text.

## New prompt

In RAG we need to add another marker to indicate where the new information (or context) should be inserted for this we use the variable named `{context}`.

In [61]:
prompt= """<start_of_turn>
Use the following pieces of context to answer the question at the end.
Don't try to make up an answer and only use the information you know.
Use three sentences maximum and keep the answer as concise as possible.
You must answer in english.
{context}<end_of_turn>
<start_of_turn>{question}<end_of_turn>"""


prompt_template = PromptTemplate(input_variables=["context", "question"],template=prompt)

## Creating the chain

In [78]:
# Top k of chunks to retrieve from the vectorstore
NB_RETRIVED_CHUNKS = 8

In [79]:
rqa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": NB_RETRIVED_CHUNKS}),
        chain_type_kwargs={"prompt": prompt_template},
        input_key="question", # same as the variable in the prompt
        output_key="answer",
        return_source_documents=True,
    )

## Chatting with a pdf

In [86]:
result = rqa_chain.invoke("Can I fly to Graubunden and is it there a lot of sunshine ?")
result

{'question': 'Can I fly to Graubunden and is it there a lot of sunshine ?',
 'answer': 'Graubunden can be reached by flight, with Zurich Airport being conveniently connected to the public transport network. The region enjoys plenty of sunshine, making it ideal for outdoor activities and a warm climate.',
 'source_documents': [Document(page_content='24Tips and Information \nCulture and leisure-time activities\nwww.graubuenden.ch\nWeather forecasts\nwww.wetter-graubuenden.ch \nPublic transport\nwww.rhb.ch | www.sbb.ch4. QUALITY OF LIFE AND LIFESTYLE\nHealth and Safety\nQuality of life: plenty of sunshine, no fog\nGood weather and a favourable climate improve wellbeing. \nThe Grisons is one of the areas with the most sunshine in \nSwitzerland and thanks to the long hours of sun and the \nmild climate, it is also the warmest vine growing area in the', metadata={'source': 'data/PDFs/Quality_of_life_in_the_Grisons.pdf', 'page': 4}),
  Document(page_content='How to get there\nFlying to Graubu

## Embellishing the output

In [87]:
def prepare_document(x):
        return x if x is None else os.path.basename(x)

def prepare_page(x):
        return x if x is None else int(x) + 1

def prepare_source(x):
        return {
            "document": prepare_document(x.metadata.get("source", None)),
            "page": prepare_page(x.metadata.get("page", None)),
            "chunk": x.page_content,
        }

In [88]:
console.print(result.get("answer"))

Graubunden can be reached by flight, with Zurich Airport being conveniently connected to the public transport 
network. The region enjoys plenty of sunshine, making it ideal for outdoor activities and a warm climate.

In [89]:
sources = [prepare_source(x) for x in result["source_documents"]]
console.print(json.dumps(sources, indent=1),highlight=False)

[
 {
  "document": "Quality_of_life_in_the_Grisons.pdf",
  "page": 5,
  "chunk": "24Tips and Information \nCulture and leisure-time activities\nwww.graubuenden.ch\nWeather 
forecasts\nwww.wetter-graubuenden.ch \nPublic transport\nwww.rhb.ch | www.sbb.ch4. QUALITY OF LIFE AND 
LIFESTYLE\nHealth and Safety\nQuality of life: plenty of sunshine, no fog\nGood weather and a favourable climate 
improve wellbeing. \nThe Grisons is one of the areas with the most sunshine in \nSwitzerland and thanks to the long
hours of sun and the \nmild climate, it is also the warmest vine growing area in the"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 5,
  "chunk": "How to get there\nFlying to Graubunden\nZurich Airport is one of Europe\u2019s air traffic  hubs and is
served by numerous \nairlines. Additionally, it is ideally connected to the  public transport network, 
\nguaranteeing a  seamless post-flight journey to  Graubunden.\nNumerous airports in and around  Graubunden include
Friedrichshafen (DE),  \nInnsbruck (AUT), St. Gallen-Altenrhein,  Basel, and Samedan near St. Moritz.\n \nDriving 
to Graubunden"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 1,
  "chunk": "THE NO.1 HOLIDAY DESTINATION  \nIN SWITZERLANDVisit  \nGraubunden\nTravel Brochure\nPhoto by\n\u00a9 
Flims Laax Falera / Gaudenz Danuser"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 2,
  "chunk": "Welcome to Graubunden\nPerfect for families seeking spectacular outdoor\nadventure, each region of the 
canton is only a\nhop, skip, and a jump away.  \nTaking the title for the easternmost and largest\ncanton of 
Switzerland - bordering Austria, Italy,\nand Liechtenstein - when asked what makes this\nplace so special, locals 
wax lyrical about its wild,\nnatural beauty coupled with the finer things in life.\nA place of bright green 
valleys, snow-capped\npeaks and shimmering Alpine lakes, days are"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 12,
  "chunk": "23 22Luxury\nRenowned for its world-\nclass health spas, elegant\nresorts, Michelin-starred 
\nrestaurants and famed shopping \nboulevards, life is beautiful in \nGraubunden. \nHighlights include:\n\u2022 
Grand Resort Bad Ragaz\n\u2022 7132 Silver Restaurant\n\u2022 Glacier Express  \n  Excellence Class\n\u2022 Via 
Serlas St. Moritz\nPhoto by\n\u00a9 Grand Resort Bad Ragaz\nGlacier Express \nExcellence Class7132 SilverVia Serlas
\nSt. Moritz\nPhoto by\n\u00a9 Kulm Hotel St. Moritz"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 2,
  "chunk": "been titled among the most spectacular train\njourneys in the world.Photo by\n\u00a9 \u00a9 Daniel 
GeigerEven with Switzerland\u2019s competitive repertoire\nof awe-inspiring scenery, delicious cheese 
and\nchocolate, world-class medical spas, and leading\nboarding schools, the canton (state) of Graubunden \nstands 
out. Renowned as one of the grandest of the \nSwiss Alpine Valleys, its supermodel looks are home \nto the classic 
children\u2019s novel, Heidi, the sparkly ski \nresorts of St. Moritz, and the shimmering Lake"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 6,
  "chunk": "Photo by\n\u00a9 Graubunden Tourism\nArosa Barenland Toboggan Run Globi Hiking TrailFamily\nSafe, 
beautiful and easy to  \ntraverse, Graubunden is a dream \nfor family travel. Kids will revel in \nrushing down 
Switzerland\u2019s  \nlongest toboggan run, reliving \nchildhood memories in Heidiland, \nor making friends with 
Globi,  \nSwitzerland\u2019s Mickey Mouse. \nHighlights include:\n\u2022 Heidi Village\n\u2022 Toboggan run\n\u2022
Madrisa Land\n\u2022 Arosa Barenland\n\u2022 Globi Hiking Trail\n11 10Photo by\n\u00a9 Graubunden Tourism"
 },
 {
  "document": "2023-travel-brochure-visit-graubunden-gcc.pdf",
  "page": 5,
  "chunk": "Driving to Graubunden\nIf you are coming from another European  destinatio